# Compare ResNet Model
- Compare Keras vs onnx runtime

## ResNet pretrained 모델 받기
- Keras로 ResNet pretrained 모델 생성 
- imagenet 학습된 가중치 사용, test 는 일반 이미지 1장으로
- <b> .h5 모델 로드 후 .onnx로 변환 </b>


In [1]:
# keras to onnx
import os
os.environ['TF_KERAS'] = '1'

# load the model
from tensorflow.keras.applications.resnet50 import ResNet50
import numpy as np

model = ResNet50(weights='imagenet')
model.summary()

# 모델 저장
model.save('model/ResNet.h5')
print('저장완료')

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

저장완료


## Keras model to onnx
- .h5 to .onnx

In [10]:
# keras to onnx
import os
os.environ['TF_KERAS'] = '1'

# keras to onnx
import onnx
from keras2onnx import convert_keras
from tensorflow.keras.models import load_model
import onnxruntime as rt
import numpy as np

model = load_model('model/ResNet.h5')
onx = convert_keras(model, 'ResNet.onnx')

with open("model/ResNet.onnx", "wb") as f:
    f.write(onx.SerializeToString())
    
print('완료')

tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 458 -> 127


완료


## Image 전처리

In [14]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

# load an image from file
image = load_img('Sannosawa1.jpg')

# convert the image pixels to a numpy array
image = img_to_array(image)

from skimage.transform import resize

image = resize(image / 255, (224, 224, 3), anti_aliasing=True)
image = image[np.newaxis, :, :, :] 
image = image.astype(np.float32)

print("new shape:", image.shape)

new shape: (1, 224, 224, 3)


## Inference time 비교

In [17]:
import time

# Keras
# 시간측정
start = time.time()
for i in range(10):
    yhat = model.predict(image)
end = time.time()
print('keras - resnet50 걸린시간:', np.double(end-start))

# ONNX
# Compute the prediction with ONNX Runtime
sess = rt.InferenceSession('model/ResNet.onnx')
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name

# 시간측정
start = time.time()
for i in range(10):
    pred_onx = sess.run([label_name], {input_name: image.astype(np.float32)})[0]
end = time.time()

print('onnxruntime - ResNet50 걸린시간:', np.double(end-start))

keras - resnet50 걸린시간: 1.4461338520050049
onnxruntime - ResNet50 걸린시간: 0.5495290756225586
